In [2]:
import carla
import random
import queue
import numpy as np
import cv2


import torch
import math
# import torchvision.transforms as transforms
from typing import *

import glob
import os
import sys
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass
import carla

import random
import time
import numpy as np
import cv2

class CarlaPatch:
    """
    init parameters:
        flat_patch: torch.Tensor or ndarray, with shape (C, H, W)
        patch_location: tuple or carla.Location, (x, y, z), e.g. (10, 10.1, 0.6)
        patch_rotation: tuple or carla.Rotation, (pitch, yaw, roll), e.g. (30 degrees, 20, 10)
        pixel_length: float [对应的是patch的一个像素在仿真世界里对应的长度，这个数值大概多少我还没有概念， 需要尝试一下]
    """
    def __init__(
            self,
            flat_patch: torch.Tensor or np.ndarray,
            patch_location: Tuple or carla.Location,
            patch_rotation: Tuple or carla.Rotation,
            pixel_length: float = 0.1,
            patch_transform: Callable = None

    ):
        self.flat_patch = flat_patch.cpu() if isinstance(flat_patch, torch.Tensor) else torch.tensor(flat_patch)

        self.patch_location = patch_location
        self.patch_rotation = patch_rotation
        self.pixel_length = pixel_length

        self.patch_pos_3d = self._trans_all_dots2pos_3d()

    @staticmethod
    def _default_patch_trans(patch: torch.Tensor):
        pass

    def _trans_all_dots2pos_3d(self) -> torch.Tensor:
        """
        通过给定的location，rotation和输入的patch，返回每个像素点的坐标，形状是（3， H， W），3对应x, y, z坐标
        """
        patch_height, patch_width = self.flat_patch.shape[1], self.flat_patch.shape[2]

        x_init_coords = torch.arange(0, self.pixel_length * patch_height, self.pixel_length)
        y_init_coords = torch.arange(0, self.pixel_length * patch_width, self.pixel_length)
        z_init_coords = torch.zeros((patch_height, patch_width))
        patch_init_coords = torch.stack([
            x_init_coords.unsqueeze(-1).broadcast_to(-1, patch_width),
            y_init_coords.unsqueeze(0).broadcast_to(patch_height, -1),
            z_init_coords,
        ], dim=0)

        if isinstance(self.patch_location, carla.Location):
            x, y, z = self.patch_location.x, self.patch_location.y, self.patch_location.z
        else:
            x, y, z = self.patch_location[0], self.patch_location[1], self.patch_location[2]

        location = torch.tensor([x, y, z]).unflatten(0, (3, 1, 1)).broadcast_to((3, patch_height, patch_width))

        if isinstance(self.patch_rotation, carla.Rotation):
            pitch, yaw, roll = self.patch_rotation.pitch, self.patch_rotation.yaw, self.patch_rotation.roll
        else:
            pitch, yaw, roll = self.patch_rotation[0], self.patch_rotation[1], self.patch_rotation[2]

        pitch = pitch / 180.0 * math.pi
        yaw = yaw / 180.0 * math.pi
        roll = roll / 180.0 * math.pi

        rx = torch.tensor([
            [1, 0, 0], [0, math.cos(roll), - math.sin(roll)], [0, math.sin(roll), math.cos(roll)],
        ], dtype=torch.float)
        ry = torch.tensor([
            [math.cos(pitch), 0, math.sin(pitch)], [0, 1, 0], [- math.sin(pitch), 0, math.cos(pitch)]
        ], dtype=torch.float)
        rz = torch.tensor([
            [math.cos(yaw), - math.sin(yaw), 0], [math.sin(yaw),  math.cos(yaw), 0], [0, 0, 1]
        ], dtype=torch.float)


        coords_rotated = rz @ ry @ rx @ patch_init_coords.reshape(3, -1)

        coords = coords_rotated.reshape(3, patch_height, patch_width) + location

        return torch.concat([coords, torch.ones((1, patch_height, patch_width))], dim=0)

    def get_patch(self):
        """
        返回patch本身以及patch每个像素点的3d坐标
        """
        return self.flat_patch, self.patch_pos_3d

    def __call__(self, *args, **kwargs):
        return self.get_patch()


class CameraCallback:
    """
    init_parameters:
        rgb_camera: Sensor in carla
        carla_patch: instance of CarlaPatch
        output_folder_path: path of folder to save camera outputs(image format)
    """
    def __init__(self, rgb_camera: carla.Sensor, camera_bp, vehicle, carla_patch: CarlaPatch, output_folder_path=None, depth_camera=None):
        self.rgb_camera = rgb_camera
        self.camera_bp = camera_bp
        self.carla_patch = carla_patch
        self.patch_data, self.patch_global_coords = self.carla_patch()
        self.folder_path = output_folder_path
        self.vehicle = vehicle

    def _get_matrices(self, camera_data):
        fov = self.camera_bp.get_attribute('fov').as_float()
        focal = camera_data.width / (2.0 * np.tan(fov * np.pi / 360.0))
        in_mat = np.identity(3)
        in_mat[0, 0] = focal / 1.00
        in_mat[1, 1] = focal / 1.00
        in_mat[0, 2] = camera_data.width / 2.0
        in_mat[1, 2] = camera_data.height / 2.0

        ex_mat = self.rgb_camera.get_transform().get_inverse_matrix()

        return torch.tensor(in_mat, dtype=torch.float32), torch.tensor(ex_mat)

    @staticmethod
    def _convert_raw_data(camera_data):
        camera_data.convert(carla.ColorConverter.Raw)
        img_tensor = torch.tensor(camera_data.raw_data, dtype=torch.uint8)
        # convert to (c, h, w)
        return img_tensor.reshape((camera_data.height, camera_data.width, 4))[:, :, :3].permute((2, 0, 1))

    def __call__(self, camera_data: carla.Image):
        in_mat, ex_mat = self._get_matrices(camera_data)

        patch_camera_coords = (torch.tensor([[0, 1, 0, 0], [0, 0, -1, 0], [1, 0, 0.0, 0]]) @ ex_mat @ self.patch_global_coords.reshape(4, -1))[:3, :]

        patch_pos2d = in_mat @ patch_camera_coords

        # normalize coords
        patch_pos2d[0] /= patch_pos2d[2]
        patch_pos2d[1] /= patch_pos2d[2]

        patch_pos2d = patch_pos2d.reshape(self.patch_data.shape)

        indices = patch_pos2d[0: 2].to(torch.long)
        # TODO: add distance mask when using depth camera
        mask = (indices[0] >= 0) & (indices[1] >= 0) & (indices[0] < camera_data.width) & (indices[1] < camera_data.height)
        zeros_indices = torch.zeros_like(indices[0])
        indices = mask * indices + ~mask * zeros_indices

        img_tensor = CameraCallback._convert_raw_data(camera_data)

        patch_point = self.patch_global_coords[:3, 0, 0]
        ray = carla.Location(patch_point[0].item(), patch_point[1].item(), patch_point[2].item()) - self.rgb_camera.get_transform().location
        forward_vec = self.rgb_camera.get_transform().get_forward_vector()

        if  forward_vec.dot(ray) > 0:
            # replace pixels in camera output with patch pixels
            img_tensor[:, indices[1], indices[0]] = (self.patch_data * 255).to(torch.uint8)

        # save camera output to disk
        cv2.imwrite(self.folder_path + f'{camera_data.frame}.jpg', img_tensor.permute(1, 2, 0).numpy())


def get_camera2world_matrix(carla_transform: carla.Transform, real_y_axis=True):
    """
    Args:
        carla_transform: Carla.Transform instance, contains carla.Location and carla.Rotation
        real_y_axis: return real y-axis value when setting true. but the view of point cloud in open-3d
            will be reversed in yaw direction.
    Returns:
        a 4x4 rotation & transaction matrix that transforms coords from camera coord-sys to simu-world coord-sys.
    """
    camera2vehicle_matrix = np.array([[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1]], dtype=np.float64)

    pitch = carla_transform.rotation.pitch / 180.0 * math.pi
    yaw = carla_transform.rotation.yaw / 180.0 * math.pi
    roll = carla_transform.rotation.roll / 180.0 * math.pi
    loc_x = carla_transform.location.x
    loc_y = - carla_transform.location.y
    loc_z = carla_transform.location.z
    sin_y, sin_p, sin_r = math.sin(yaw), math.sin(pitch), math.sin(roll)
    cos_y, cos_p, cos_r = math.cos(yaw), math.cos(pitch), math.cos(roll)

    vehicle2world_matrix = np.array([
        [cos_y * cos_p, cos_y * sin_p * sin_r + sin_y * cos_r, - cos_y * sin_p * cos_r + sin_y * sin_r, loc_x],
        [-sin_y * cos_p, - sin_y * sin_p * sin_r + cos_y * cos_r, sin_y * sin_p * cos_r + cos_y * sin_r, loc_y],
        [sin_p, -cos_p * sin_r, cos_p * cos_r, loc_z],
        [0.0, 0.0, 0.0, 1.0]
    ])
    if real_y_axis:
        vehicle2world_matrix[2] *= -1

    return vehicle2world_matrix @ camera2vehicle_matrix


#构造相机投影矩阵函数
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(loc, K, w2c):
    # 计算三维坐标的二维投影

    # 格式化输入坐标（loc 是一个 carla.Position 对象）
    point = np.array([loc.x, loc.y, loc.z, 1])

    # 转换到相机坐标系
    point_camera = np.dot(w2c, point)

    # 将坐标系从 UE4 的坐标系转换为标准坐标系（y, -z, x），同时移除第四个分量
    # point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

    # 使用相机矩阵进行三维到二维投影
    point_img = np.dot(K, point_camera[:3])

    # 归一化
    point_img[0] /= point_img[2]
    point_img[1] /= point_img[2]

    return point_img[0:2]

#连接Carla并获取世界
client = carla.Client('localhost', 2000)
world  = client.get_world()
bp_lib = world.get_blueprint_library()

# 生成车辆
vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
spawn_points = world.get_map().get_spawn_points()
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
print(spawn_points[0].location.x, spawn_points[0].location.y, spawn_points[0].location.z)

# 生成相机
camera_bp = bp_lib.find('sensor.camera.rgb')

camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
vehicle.set_autopilot(True)

#生成目标车辆
for i in range(20):
    vehicle_bp = random.choice(bp_lib.filter('vehicle'))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
    if npc:
        npc.set_autopilot(True)

# 设置仿真模式为同步模式
settings = world.get_settings()
settings.synchronous_mode = True # 启用同步模式
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# 创建对接接收相机数据
image_queue = queue.Queue()
camera.listen(image_queue.put)

# 从相机获取属性
image_w = camera_bp.get_attribute("image_size_x").as_int()  # 图像宽度
image_h = camera_bp.get_attribute("image_size_y").as_int()  # 图像高度
fov = camera_bp.get_attribute("fov").as_float()  # 视场角

# 计算相机投影矩阵，用于从三维坐标投影到二维坐标
K = build_projection_matrix(image_w, image_h, fov)

edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]

# 获取第一张图像
world.tick()
image = image_queue.get()

# 将原始数据重新整形为 RGB 数组
img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# 在 OpenCV 的显示窗口中显示图像
cv2.namedWindow('ImageWindowName', cv2.WINDOW_AUTOSIZE)
cv2.imshow('ImageWindowName', img)
cv2.waitKey(1)

patch = torch.load('C:/Users/Kai/Desktop/patch.pt', map_location='cpu')
carla_patch = CarlaPatch(patch, patch_location=(200, 100, 35), patch_rotation=(70, 60, 40), pixel_length=0.5)

patch_data, coords_3d = carla_patch()
patch_data.detach_()
patch_data  = (patch_data * 255).to(torch.uint8)

while True:
    # 更新世界状态并获取图像
    world.tick()
    image = image_queue.get()

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    # img = np.flip(img,(0,))
    # img = np.transpose(img, (1, 0, 2))

    transform = camera.get_transform()

    # point = coords_3d.reshape(4, -1).numpy()
    # # 转换到相机坐标系
    #
    # point_camera = (np.linalg.inv(get_camera2world_matrix(transform, real_y_axis=True)) @ point)[:3]
    # # 将坐标系从 UE4 的坐标系转换为标准坐标系（y, -z, x），同时移除第四个分量
    # # point_camera = [point_camera[1], point_camera[2], point_camera[0]]
    # # 使用相机矩阵进行三维到二维投影
    # point_img = K @ point_camera
    # point_img[0] /= point_img[2]
    # point_img[1] /= point_img[2]
    #
    #
    # point_img = np.array(point_img[0:2].reshape(2, 128, 128), dtype=np.int64)
    # mask = (point_img[0] >= 0) & (point_img[1] >= 0) & (point_img[0] < 800) & (point_img[1] < 600)
    # zeros_indices = np.zeros_like(point_img[0])
    # point_img = mask * point_img + ~mask * zeros_indices
    #
    # ray = carla.Location(coords_3d[:3, 0, 0][0].item(), coords_3d[:3, 0, 0][1].item(), coords_3d[:3, 0, 0][2].item()) - vehicle.get_transform().location
    # forward_vec = vehicle.get_transform().get_forward_vector()
    #
    # if  forward_vec.dot(ray) > 1:
    #     # replace pixels in camera output with patch pixels
    #     img[point_img[0], point_img[1], :3] = patch_data.permute((1, 2, 0)).numpy()

    # for loop format
    #
    # for i in range(128):
    #     for j in range(128):
    #         posi = coords_3d[:, i, j]
    #         p = get_image_point(carla.Location(posi[0].item(), posi[1].item(), posi[2].item()), K, world_2_camera)
    #         if 0 <= p[0] < 600 and 0 <= p[1] < 800:
    #             img[int(p[1]), int(p[0]), :3] = patch_data[:, i, j].numpy()

    world_2_camera = np.linalg.inv(get_camera2world_matrix(transform, real_y_axis=False))

    for npc in world.get_actors().filter('*vehicle*'):
        # 过滤掉自车
        if npc.id != vehicle.id:
            bb = npc.bounding_box
            dist = npc.get_transform().location.distance(vehicle.get_transform().location)

            # 筛选距离在50米以内的车辆
            if dist < 50:
                forward_vec = vehicle.get_transform().get_forward_vector()
                ray = npc.get_transform().location - vehicle.get_transform().location

                # 计算车辆前进方向与车辆之间的向量的点积，
                # 通过阈值判断是否在相机前方绘制边界框
                if forward_vec.dot(ray) > 1:

                    p1 = get_image_point(bb.location, K, world_2_camera)
                    verts = [v for v in bb.get_world_vertices(npc.get_transform())]

                    for edge in edges:
                        # print('bb location', (verts[edge[0]].x, verts[edge[0]].y, verts[edge[0]].z))
                        # print('self location:', vehicle.get_transform().location.x, vehicle.get_transform().location.y, vehicle.get_transform().location.z)
                        p1 = get_image_point(verts[edge[0]], K, world_2_camera)
                        p2 = get_image_point(verts[edge[1]], K, world_2_camera)
                        cv2.line(img, (int(p1[0]), int(p1[1])), (int(p2[0]), int(p2[1])), (255, 0, 0, 255), 1)

    cv2.imshow('ImageWindowName', img)
    if cv2.waitKey(1) == ord('q'):
        break


settings.synchronous_mode = False
world.apply_settings(settings)
camera.destroy()
image_queue.queue.clear()
vehicle.destroy()
cv2.destroyAllWindows()

-64.64484405517578 24.471010208129883 0.5999999642372131


In [21]:
coords_3d[:, 127, 127]

tensor([187.9101, 176.3473, -10.7102,   1.0000])

In [14]:
point_img[:, 100].max()

285

In [ ]:
patch = torch.load('C:/Users/Kai/Desktop/patch.pt', map_location='cpu')
carla_patch = CarlaPatch(patch, patch_location=(-64, -24, 10), patch_rotation=(70, 60, 40), pixel_length=0.8)

patch_data, coords_3d = carla_patch()
coords_3d[:, 0, 0]

In [ ]:
import sys
sys.path.append('utils')
import cv2
import numpy as np
import torch
# from utils.utils import *
# from utils.visualize_utils import *
import random
import carla
# from carla_patch import *

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(500.0)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
spawnpoints = world.get_map().get_spawn_points()
spectator = world.get_spectator()

In [ ]:
sp = spawnpoints[0]
(sp.location.x, sp.location.y, sp.location.z)

In [ ]:
vehicle_blueprint = blueprint_library.find('vehicle.audi.etron')
#Fahrzeug wird an einem zufälligen Spawnpunkt gespawnt
vehicle = world.try_spawn_actor(vehicle_blueprint, spawnpoints[0])
vehicle.set_autopilot(True)
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-2,z=1.5)),vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [ ]:
# settings = world.get_settings()
# settings.synchronous_mode = True # 启用同步模式
# settings.fixed_delta_seconds = 0.05
# world.apply_settings(settings)

In [ ]:
camera_blueprint = blueprint_library.find('sensor.camera.rgb')
# camera_blueprint.set_attribute('focal_distance', '1000')
camera_transform = carla.Transform(carla.Location(z=2,x=0.5))
camera = world.spawn_actor(camera_blueprint,camera_transform,attach_to=vehicle)

In [ ]:
patch = torch.load('C:/Users/Kai/Desktop/patch.pt', map_location='cpu')
carla_patch = CarlaPatch(patch, patch_location=(200, -250, 16), patch_rotation=(70, 60, 40), pixel_length=0.5)

In [ ]:
camera_callback = CameraCallback(camera, camera_blueprint, vehicle, carla_patch, output_folder_path='C:/Users/Kai/Desktop/frames/')

In [ ]:
camera.listen(lambda image: camera_callback(image))

In [ ]:
camera.destroy()

In [ ]:
def _get_matrices(camera_bp, rgb_camera):
    fov = camera_bp.get_attribute('fov').as_float()
    focal = camera_blueprint.get_attribute('image_size_x').as_int() / (2.0 * np.tan(fov * np.pi / 360.0))
    in_mat = np.identity(3)
    in_mat[0, 0] = focal / 1.00
    in_mat[1, 1] = focal / 1.00
    in_mat[0, 2] = camera_blueprint.get_attribute('image_size_x').as_int() / 2.0
    in_mat[1, 2] = camera_blueprint.get_attribute('image_size_y').as_int() / 2.0

    ex_mat = rgb_camera.get_transform().get_inverse_matrix()

    return torch.tensor(in_mat, dtype=torch.float32), torch.tensor(ex_mat)


In [ ]:
vehicle.set_autopilot(False)

In [ ]:
in_mat, ex_mat = _get_matrices(camera_blueprint, camera)
camera.get_transform().location.x, camera.get_transform().location.y, camera.get_transform().location.z

In [ ]:
coords_camera = ex_mat @ torch.tensor([-100.0, 75, 2, 1])
coords_camera

In [ ]:
swap_axis_coords = torch.tensor([coords_camera[1], - coords_camera[2], coords_camera[0]])
swap_axis_coords

In [ ]:
coords_pix = (in_mat @ swap_axis_coords)
coords_pix

In [ ]:
coords_pix[0] /= coords_pix[2]
coords_pix[1] /= coords_pix[2]
coords_pix.to(int)

In [ ]:
import sys
sys.path.append('utils')
from utils.visualize_utils import *

In [ ]:
import torch
import torchvision.transforms as transforms

In [ ]:
patch = torch.load('C:/Users/Kai/Desktop/patch.pt', map_location='cpu')

In [ ]:
show(patch)

In [ ]:
show(patch.transpose(1, 2))

In [ ]:
show(transforms.RandomVerticalFlip(p=1)(transforms.RandomHorizontalFlip(p=1)(patch)))

In [ ]:
import numpy as np
array = np.ndarray((600, 800, 3))

In [ ]:
np.flip(array, axis=(1)).shape

In [ ]:
array[-3]